In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
from MyCreds.mycreds import Capstone_AWS_PG
from sqlalchemy import create_engine
import numpy as np

In [2]:
engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

# Switch depending on which table you want to read.
# sql_query = 'select * from resale_prices_based_on_registration_date;'
sql_query = 'select * from resale_prices_based_on_approval_date;'

with engine.connect() as cnxn:
    approval = pd.read_sql(sql_query, cnxn)


approval

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,n_rooms,storey_range_min,storey_range_max
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN,NaN,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN,NaN,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN,NaN,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN,NaN,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313689,2012-02-01,GEYLANG,4 ROOM,319,UBI AVE 1,10 TO 12,84.0,Simplified,1985,383000.0,NaN,10.0,12.0
1313690,2012-02-01,GEYLANG,4 ROOM,314,UBI AVE 1,01 TO 03,84.0,Simplified,1985,382000.0,NaN,1.0,3.0
1313691,2012-02-01,GEYLANG,4 ROOM,343,UBI AVE 1,01 TO 03,84.0,Simplified,1986,380000.0,NaN,1.0,3.0
1313692,2012-02-01,HOUGANG,3 ROOM,631,HOUGANG AVE 8,07 TO 09,60.0,Improved,1985,300000.0,NaN,7.0,9.0


In [3]:
def clean_approval_df(df):

    df['month'] = pd.to_datetime(df['month'])
    df['n_rooms'] = df['flat_type'].str.extract('(\d)').astype('Int64')
    df['n_rooms'] = df['n_rooms'].astype('Int64')
    # Convert story range to min and max int columns
    df['storey_range_min'] = df['storey_range'].str[:2].astype(int)
    df['storey_range_max'] = df['storey_range'].str[-2:].astype(int)

    df.sort_values('month', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df

approval = clean_approval_df(approval)
approval

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,n_rooms,storey_range_min,storey_range_max
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,1,10,12
1,1990-01-01,YISHUN,5 ROOM,229,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,98700.0,5,1,3
2,1990-01-01,YISHUN,5 ROOM,231,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,94000.0,5,1,3
3,1990-01-01,YISHUN,5 ROOM,221,YISHUN ST 21,07 TO 09,121.0,IMPROVED,1985,98000.0,5,7,9
4,1990-01-01,YISHUN,5 ROOM,161,YISHUN ST 11,10 TO 12,121.0,IMPROVED,1985,120000.0,5,10,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313689,2012-02-01,SENGKANG,4 ROOM,268A,COMPASSVALE LINK,16 TO 18,90.0,Premium Apartment,2006,525000.0,4,16,18
1313690,2012-02-01,SENGKANG,4 ROOM,268D,COMPASSVALE LINK,13 TO 15,90.0,Premium Apartment,2006,535000.0,4,13,15
1313691,2012-02-01,SENGKANG,4 ROOM,269D,COMPASSVALE LINK,16 TO 18,90.0,Premium Apartment,2006,563000.0,4,16,18
1313692,2012-02-01,SENGKANG,4 ROOM,184C,RIVERVALE CRES,16 TO 18,86.0,Model A2,2003,423000.0,4,16,18


In [4]:
# engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)
#
# # Switch depending on which table you want to read.
# sql_query = 'select * from resale_prices_based_on_registration_date;'
# # sql_query = 'select * from resale_prices_based_on_approval_date;'
#
# with engine.connect() as cnxn:
#     registration = pd.read_sql(sql_query, cnxn)


In [5]:
def create_resale_registration():

    base_dir = '../data/resale-flat-prices/'
    df = pd.DataFrame()
    for file in os.listdir('../data/resale-flat-prices/'):
        if "registration-date" in file and 'csv' in file:
            df = pd.concat([df, pd.read_csv(f'{base_dir}{file}')])

    return df

registration = create_resale_registration()
registration

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,NaN
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,NaN
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,NaN
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,NaN
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
37148,2016-12,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,488000.0,82
37149,2016-12,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,455000.0,69
37150,2016-12,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,778000.0,74
37151,2016-12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,575000.0,70


In [6]:
def clean_registration_df(df):

    df['month'] = pd.to_datetime(df['month'])
    df['n_rooms'] = df['flat_type'].str.extract('(\d)').astype('Int64')
    df['n_rooms'] = df['n_rooms'].astype('Int64')
    # Convert story range to min and max int columns
    df['storey_range_min'] = df['storey_range'].str[:2].astype(int)
    df['storey_range_max'] = df['storey_range'].str[-2:].astype(int)

    # Convert remaining lease years to int years and int months remaining columns
    df['remaining_lease_years'] = df['remaining_lease'].str.extract('(?P<remaining_lease_years>\d{2}) year[s]?')

    df[['remaining_lease_months', 'remaining_lease_month']] = df['remaining_lease'].str.extract('\d{2} year[s]? (?P<remaining_lease_months>\d{2}) month[s]?|(?P<remaining_lease_month>\d{2}) month[s]?')
    df.drop('remaining_lease_month', axis=1, inplace=True)
    df[['remaining_lease_years', 'remaining_lease_months']] = df[['remaining_lease_years', 'remaining_lease_months']].astype('Int64').copy()
    df['remaining_lease_years'].fillna(df['remaining_lease'], inplace=True)
    df.sort_values('month', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)
    df['remaining_lease_months'].fillna(pd.Series(np.where(df['remaining_lease'].isnull(), np.nan, 0)), inplace=True)



    return df

In [7]:
registration = clean_registration_df(registration)
registration

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,n_rooms,storey_range_min,storey_range_max,remaining_lease_years,remaining_lease_months
0,2012-03-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,NaN,2,6,10,<NA>,<NA>
1,2012-03-01,SENGKANG,4 ROOM,184B,RIVERVALE CRES,11 TO 15,90.0,Model A,2003,436000.0,NaN,4,11,15,<NA>,<NA>
2,2012-03-01,SENGKANG,4 ROOM,201B,COMPASSVALE DR,11 TO 15,90.0,Model A,2001,437000.0,NaN,4,11,15,<NA>,<NA>
3,2012-03-01,SENGKANG,4 ROOM,311B,ANCHORVALE LANE,01 TO 05,95.0,Premium Apartment,2002,438000.0,NaN,4,1,5,<NA>,<NA>
4,2012-03-01,SENGKANG,4 ROOM,259B,COMPASSVALE RD,01 TO 05,85.0,Model A2,2001,438000.0,NaN,4,1,5,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210825,2022-03-01,CLEMENTI,3 ROOM,208B,CLEMENTI AVE 6,19 TO 21,68.0,Model A,2017,575000.0,94 years 08 months,3,19,21,94,8
210826,2022-03-01,CLEMENTI,3 ROOM,320,CLEMENTI AVE 4,04 TO 06,67.0,New Generation,1979,380888.0,56 years 02 months,3,4,6,56,2
210827,2022-03-01,CLEMENTI,3 ROOM,334,CLEMENTI AVE 2,07 TO 09,67.0,New Generation,1978,416000.0,55 years 05 months,3,7,9,55,5
210828,2022-03-01,GEYLANG,3 ROOM,3,JOO CHIAT RD,10 TO 12,74.0,Model A,1985,470000.0,62 years 10 months,3,10,12,62,10


In [8]:
def create_resale_prices(approval_df, registration_df):
    rs_df = pd.concat([approval_df, registration_df])

    rs_df['month'] = pd.to_datetime(rs_df['month'])
    rs_df['lease_commence_date'] = pd.to_datetime(rs_df['lease_commence_date'], format='%Y')

    rs_df['id'] = rs_df['month'].dt.strftime('%Y') + "_" + rs_df['month'].dt.strftime('%m') + "_" +  rs_df['block'] + "_" + rs_df['street_name'].str.replace(' ', '_') + "_" + rs_df['storey_range'].str.replace(' ', '_')

    # Remaining lease years =
    # if month >= 6
    # lease_commence_date + 99 - month
    # if month < 6
    # lease_commence_date + 98 - month
    rs_df['remaining_lease_years'] = np.where(rs_df['month'].dt.month >= 6, ((rs_df['lease_commence_date'].dt.year + 98) - rs_df['month'].dt.year), ((rs_df['lease_commence_date'].dt.year + 99) - rs_df['month'].dt.year))

    # Convert back to int year.
    rs_df['lease_commence_date'] = rs_df['lease_commence_date'].dt.year.astype(int)


    # Price per square foot and meter
    rs_df['price_per_sq_ft'] = rs_df['resale_price'] / (rs_df['floor_area_sqm'] * 3.28084) #3.28084 ft per meter
    rs_df['price_per_sq_m'] = rs_df['resale_price'] / rs_df['floor_area_sqm']

    # Price per square foot/meter and remaining lease years
    rs_df['price_per_sq_ft_per_lease_yr'] = rs_df['price_per_sq_ft'] / rs_df['remaining_lease_years']
    rs_df['price_per_sq_m_per_lease_yr'] = rs_df['price_per_sq_m'] / rs_df['remaining_lease_years']



    return rs_df

rs_df = create_resale_prices(approval, registration)
rs_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,storey_range_min,storey_range_max,remaining_lease,remaining_lease_years,remaining_lease_months,id,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,...,10,12,NaN,86,<NA>,1990_01_309_ANG_MO_KIO_AVE_1_10_TO_12,88.490320,290.322581,1.028957,3.375844
1,1990-01-01,YISHUN,5 ROOM,229,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,98700.0,...,1,3,NaN,94,<NA>,1990_01_229_YISHUN_ST_21_01_TO_03,248.626108,815.702479,2.644959,8.677686
2,1990-01-01,YISHUN,5 ROOM,231,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,94000.0,...,1,3,NaN,94,<NA>,1990_01_231_YISHUN_ST_21_01_TO_03,236.786769,776.859504,2.519008,8.264463
3,1990-01-01,YISHUN,5 ROOM,221,YISHUN ST 21,07 TO 09,121.0,IMPROVED,1985,98000.0,...,7,9,NaN,94,<NA>,1990_01_221_YISHUN_ST_21_07_TO_09,246.862802,809.917355,2.626200,8.616142
4,1990-01-01,YISHUN,5 ROOM,161,YISHUN ST 11,10 TO 12,121.0,IMPROVED,1985,120000.0,...,10,12,NaN,94,<NA>,1990_01_161_YISHUN_ST_11_10_TO_12,302.280982,991.735537,3.215755,10.550378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210825,2022-03-01,CLEMENTI,3 ROOM,208B,CLEMENTI AVE 6,19 TO 21,68.0,Model A,2017,575000.0,...,19,21,94 years 08 months,94,8,2022_03_208B_CLEMENTI_AVE_6_19_TO_21,2577.352859,8455.882353,27.418647,89.956195
210826,2022-03-01,CLEMENTI,3 ROOM,320,CLEMENTI AVE 4,04 TO 06,67.0,New Generation,1979,380888.0,...,4,6,56 years 02 months,56,2,2022_03_320_CLEMENTI_AVE_4_04_TO_06,1732.756100,5684.895522,30.942073,101.515991
210827,2022-03-01,CLEMENTI,3 ROOM,334,CLEMENTI AVE 2,07 TO 09,67.0,New Generation,1978,416000.0,...,7,9,55 years 05 months,55,5,2022_03_334_CLEMENTI_AVE_2_07_TO_09,1892.489492,6208.955224,34.408900,112.890095
210828,2022-03-01,GEYLANG,3 ROOM,3,JOO CHIAT RD,10 TO 12,74.0,Model A,1985,470000.0,...,10,12,62 years 10 months,62,10,2022_03_3_JOO_CHIAT_RD_10_TO_12,1935.891830,6351.351351,31.224062,102.441151


In [9]:
def update_street_abbreviations(resale_df):
    resale_df['address']=resale_df['block']+' '+resale_df['street_name']

    ##Deal with short forms
    resale_df['address']=resale_df['address'].str.replace(' AVE ',' AVENUE ')
    resale_df['address']=resale_df['address'].str.replace(' ST ',' STREET ')
    resale_df['address']=resale_df['address'].str.replace(' NTH',' NORTH')
    resale_df['address']=resale_df['address'].str.replace(' STH',' SOUTH')
    resale_df['address']=resale_df['address'].str.replace(' RD',' ROAD')
    resale_df['address']=resale_df['address'].str.replace(' UPP ',' UPPER ')
    resale_df['address']=resale_df['address'].str.replace(' CTRL',' CENTRAL')
    resale_df['address']=resale_df['address'].str.replace(' JLN ',' JALAN ')
    resale_df['address']=resale_df['address'].str.replace('BT ','BUKIT ')
    resale_df['address']=resale_df['address'].str.replace(' HTS',' HEIGHTS')
    resale_df['address']=resale_df['address'].str.replace(' MKT ',' MARKET ')
    resale_df['address']=resale_df['address'].str.replace('TG ','TANJONG ')
    resale_df['address']=resale_df['address'].str.replace("C'WEALTH",'COMMONWEALTH')
    resale_df['address']=resale_df['address'].str.replace("LOR ",'LORONG ')
    resale_df['address']=resale_df['address'].str.replace("GDNS",'GARDENS')
    resale_df['address']=resale_df['address'].str.replace('ST. ','SAINT ')
    resale_df['address']=resale_df['address'].str.replace(' PK',' PARK')
    resale_df['address']=resale_df['address'].str.replace(' DR ',' DRIVE ')
    resale_df['address']=resale_df['address'].str.replace('KG ','KAMPONG ')
    resale_df['address']=resale_df['address'].str.replace(' CL ',' CLOSE ')

    return resale_df

In [10]:
rs_df = update_street_abbreviations(rs_df)
rs_df

/var/folders/x5/8jbl553d391f5ks09z57vch40000gn/T/ipykernel_3611/2010800487.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  resale_df['address']=resale_df['address'].str.replace('ST. ','SAINT ')


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,storey_range_max,remaining_lease,remaining_lease_years,remaining_lease_months,id,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr,address
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,...,12,NaN,86,<NA>,1990_01_309_ANG_MO_KIO_AVE_1_10_TO_12,88.490320,290.322581,1.028957,3.375844,309 ANG MO KIO AVENUE 1
1,1990-01-01,YISHUN,5 ROOM,229,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,98700.0,...,3,NaN,94,<NA>,1990_01_229_YISHUN_ST_21_01_TO_03,248.626108,815.702479,2.644959,8.677686,229 YISHUN STREET 21
2,1990-01-01,YISHUN,5 ROOM,231,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,94000.0,...,3,NaN,94,<NA>,1990_01_231_YISHUN_ST_21_01_TO_03,236.786769,776.859504,2.519008,8.264463,231 YISHUN STREET 21
3,1990-01-01,YISHUN,5 ROOM,221,YISHUN ST 21,07 TO 09,121.0,IMPROVED,1985,98000.0,...,9,NaN,94,<NA>,1990_01_221_YISHUN_ST_21_07_TO_09,246.862802,809.917355,2.626200,8.616142,221 YISHUN STREET 21
4,1990-01-01,YISHUN,5 ROOM,161,YISHUN ST 11,10 TO 12,121.0,IMPROVED,1985,120000.0,...,12,NaN,94,<NA>,1990_01_161_YISHUN_ST_11_10_TO_12,302.280982,991.735537,3.215755,10.550378,161 YISHUN STREET 11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210825,2022-03-01,CLEMENTI,3 ROOM,208B,CLEMENTI AVE 6,19 TO 21,68.0,Model A,2017,575000.0,...,21,94 years 08 months,94,8,2022_03_208B_CLEMENTI_AVE_6_19_TO_21,2577.352859,8455.882353,27.418647,89.956195,208B CLEMENTI AVENUE 6
210826,2022-03-01,CLEMENTI,3 ROOM,320,CLEMENTI AVE 4,04 TO 06,67.0,New Generation,1979,380888.0,...,6,56 years 02 months,56,2,2022_03_320_CLEMENTI_AVE_4_04_TO_06,1732.756100,5684.895522,30.942073,101.515991,320 CLEMENTI AVENUE 4
210827,2022-03-01,CLEMENTI,3 ROOM,334,CLEMENTI AVE 2,07 TO 09,67.0,New Generation,1978,416000.0,...,9,55 years 05 months,55,5,2022_03_334_CLEMENTI_AVE_2_07_TO_09,1892.489492,6208.955224,34.408900,112.890095,334 CLEMENTI AVENUE 2
210828,2022-03-01,GEYLANG,3 ROOM,3,JOO CHIAT RD,10 TO 12,74.0,Model A,1985,470000.0,...,12,62 years 10 months,62,10,2022_03_3_JOO_CHIAT_RD_10_TO_12,1935.891830,6351.351351,31.224062,102.441151,3 JOO CHIAT ROAD


In [11]:
def sort_cols(resale_df):
    resale_df = resale_df[['id', 'month', 'block', 'town', 'street_name', 'address', 'storey_range', 'storey_range_min', 'storey_range_max',
      'flat_model', 'flat_type', 'n_rooms', 'floor_area_sqm',
       'lease_commence_date', 'remaining_lease', 'remaining_lease_years', 'remaining_lease_months',
      'resale_price', 'price_per_sq_ft', 'price_per_sq_m', 'price_per_sq_ft_per_lease_yr', 'price_per_sq_m_per_lease_yr']].sort_values('month', ascending=True)

    return resale_df

In [12]:
rs_df = sort_cols(rs_df)
rs_df

,id,month,block,town,street_name,address,storey_range,storey_range_min,storey_range_max,flat_model,...,floor_area_sqm,lease_commence_date,remaining_lease,remaining_lease_years,remaining_lease_months,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr
0,1990_01_309_ANG_MO_KIO_AVE_1_10_TO_12,1990-01-01,309,ANG MO KIO,ANG MO KIO AVE 1,309 ANG MO KIO AVENUE 1,10 TO 12,10,12,IMPROVED,...,31.0,1977,NaN,86,<NA>,9000.0,88.490320,290.322581,1.028957,3.375844
2004,1990_01_307_CLEMENTI_AVE_4_01_TO_03,1990-01-01,307,CLEMENTI,CLEMENTI AVE 4,307 CLEMENTI AVENUE 4,01 TO 03,1,3,NEW GENERATION,...,67.0,1979,NaN,88,<NA>,47000.0,213.814919,701.492537,2.429715,7.971506
2005,1990_01_309_CLEMENTI_AVE_4_04_TO_06,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,309 CLEMENTI AVENUE 4,04 TO 06,4,6,NEW GENERATION,...,82.0,1980,NaN,89,<NA>,57000.0,211.873164,695.121951,2.380597,7.810359
2006,1990_01_309_CLEMENTI_AVE_4_10_TO_12,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,309 CLEMENTI AVENUE 4,10 TO 12,10,12,NEW GENERATION,...,67.0,1980,NaN,89,<NA>,34000.0,154.674622,507.462687,1.737917,5.701828
2007,1990_01_310_CLEMENTI_AVE_4_04_TO_06,1990-01-01,310,CLEMENTI,CLEMENTI AVE 4,310 CLEMENTI AVENUE 4,04 TO 06,4,6,NEW GENERATION,...,67.0,1980,NaN,89,<NA>,47000.0,213.814919,701.492537,2.402415,7.881939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210529,2022_03_501B_YISHUN_ST_51_07_TO_09,2022-03-01,501B,YISHUN,YISHUN ST 51,501B YISHUN STREET 51,07 TO 09,7,9,Improved,...,112.0,2018,94 years 11 months,95,11,652000.0,1774.371372,5821.428571,18.677593,61.278195
210528,2022_03_820_YISHUN_ST_81_07_TO_09,2022-03-01,820,YISHUN,YISHUN ST 81,820 YISHUN STREET 81,07 TO 09,7,9,Improved,...,122.0,1988,65 years 06 months,65,6,585000.0,1461.540937,4795.081967,22.485245,73.770492
210527,2022_03_359_YISHUN_RING_RD_01_TO_03,2022-03-01,359,YISHUN,YISHUN RING RD,359 YISHUN RING ROAD,01 TO 03,1,3,Apartment,...,145.0,1988,65 years 05 months,65,5,682000.0,1433.610989,4703.448276,22.055554,72.360743
210538,2022_03_751_YISHUN_ST_72_04_TO_06,2022-03-01,751,YISHUN,YISHUN ST 72,751 YISHUN STREET 72,04 TO 06,4,6,New Generation,...,91.0,1984,61 years 02 months,61,2,458000.0,1534.048303,5032.967033,25.148333,82.507656


In [78]:
rs_df.dtypes

month                           datetime64[ns]
block                                   object
town                                    object
street_name                             object
address                                 object
storey_range                            object
storey_range_min                         int64
storey_range_max                         int64
flat_model                              object
flat_type                               object
n_rooms                                  Int64
floor_area_sqm                         float64
lease_commence_date                      int64
remaining_lease                         object
remaining_lease_years                    int64
remaining_lease_months                   Int64
resale_price                           float64
price_per_sq_ft                        float64
price_per_sq_m                         float64
price_per_sq_ft_per_lease_yr           float64
price_per_sq_m_per_lease_yr            float64
dtype: object

In [13]:
rs_df[rs_df['remaining_lease_years'].isnull() == True]

,id,month,block,town,street_name,address,storey_range,storey_range_min,storey_range_max,flat_model,...,floor_area_sqm,lease_commence_date,remaining_lease,remaining_lease_years,remaining_lease_months,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr


In [97]:
# from sqlalchemy import create_engine
#
# engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)
#
# sql_alc_cnxn = engine.connect()
#
# rs_df.to_sql('resale_prices', sql_alc_cnxn, if_exists='replace', index=False)
#
# sql_alc_cnxn.close()

In [14]:
# Sending to pickle for now until I can confirm normalization method for prices is accurate
rs_df.to_pickle('../data/resale-flat-prices/resale_prices.pkl')

# Resale Prices Load from Pickle

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from MyCreds.mycreds import Capstone_AWS_PG

In [3]:
rsp = pd.read_pickle('../data/resale-flat-prices/resale_prices.pkl')
rsp

,id,month,block,town,street_name,address,storey_range,storey_range_min,storey_range_max,flat_model,...,floor_area_sqm,lease_commence_date,remaining_lease,remaining_lease_years,remaining_lease_months,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr
0,1990_01_309_ANG_MO_KIO_AVE_1_10_TO_12,1990-01-01,309,ANG MO KIO,ANG MO KIO AVE 1,309 ANG MO KIO AVENUE 1,10 TO 12,10,12,IMPROVED,...,31.0,1977,NaN,86,<NA>,9000.0,88.490320,290.322581,1.028957,3.375844
2004,1990_01_307_CLEMENTI_AVE_4_01_TO_03,1990-01-01,307,CLEMENTI,CLEMENTI AVE 4,307 CLEMENTI AVENUE 4,01 TO 03,1,3,NEW GENERATION,...,67.0,1979,NaN,88,<NA>,47000.0,213.814919,701.492537,2.429715,7.971506
2005,1990_01_309_CLEMENTI_AVE_4_04_TO_06,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,309 CLEMENTI AVENUE 4,04 TO 06,4,6,NEW GENERATION,...,82.0,1980,NaN,89,<NA>,57000.0,211.873164,695.121951,2.380597,7.810359
2006,1990_01_309_CLEMENTI_AVE_4_10_TO_12,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,309 CLEMENTI AVENUE 4,10 TO 12,10,12,NEW GENERATION,...,67.0,1980,NaN,89,<NA>,34000.0,154.674622,507.462687,1.737917,5.701828
2007,1990_01_310_CLEMENTI_AVE_4_04_TO_06,1990-01-01,310,CLEMENTI,CLEMENTI AVE 4,310 CLEMENTI AVENUE 4,04 TO 06,4,6,NEW GENERATION,...,67.0,1980,NaN,89,<NA>,47000.0,213.814919,701.492537,2.402415,7.881939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210529,2022_03_501B_YISHUN_ST_51_07_TO_09,2022-03-01,501B,YISHUN,YISHUN ST 51,501B YISHUN STREET 51,07 TO 09,7,9,Improved,...,112.0,2018,94 years 11 months,95,11,652000.0,1774.371372,5821.428571,18.677593,61.278195
210528,2022_03_820_YISHUN_ST_81_07_TO_09,2022-03-01,820,YISHUN,YISHUN ST 81,820 YISHUN STREET 81,07 TO 09,7,9,Improved,...,122.0,1988,65 years 06 months,65,6,585000.0,1461.540937,4795.081967,22.485245,73.770492
210527,2022_03_359_YISHUN_RING_RD_01_TO_03,2022-03-01,359,YISHUN,YISHUN RING RD,359 YISHUN RING ROAD,01 TO 03,1,3,Apartment,...,145.0,1988,65 years 05 months,65,5,682000.0,1433.610989,4703.448276,22.055554,72.360743
210538,2022_03_751_YISHUN_ST_72_04_TO_06,2022-03-01,751,YISHUN,YISHUN ST 72,751 YISHUN STREET 72,04 TO 06,4,6,New Generation,...,91.0,1984,61 years 02 months,61,2,458000.0,1534.048303,5032.967033,25.148333,82.507656


In [22]:
rsp.columns

Index(['id', 'month', 'block', 'town', 'street_name', 'address',
       'storey_range', 'storey_range_min', 'storey_range_max', 'flat_model',
       'flat_type', 'n_rooms', 'floor_area_sqm', 'lease_commence_date',
       'remaining_lease', 'remaining_lease_years', 'remaining_lease_months',
       'resale_price', 'price_per_sq_ft', 'price_per_sq_m',
       'price_per_sq_ft_per_lease_yr', 'price_per_sq_m_per_lease_yr'],
      dtype='object')

# Housing Index Prices

In [5]:
engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

sql_alc_cnxn = engine.connect()

# Switch depending on which table you want to read.
# sql_query = 'select * from resale_price_based_on_registration_date;'
sql_query = 'select * from resale_price_index;'

hpi = pd.read_sql(sql_query, sql_alc_cnxn)
hpi

,quarter,index
0,1990-Q1,24.3
1,1990-Q2,24.4
2,1990-Q3,25.0
3,1990-Q4,24.7
4,1991-Q1,24.9
...,...,...
122,2020-Q3,133.9
123,2020-Q4,138.1
124,2021-Q1,142.2
125,2021-Q2,146.4


In [6]:
hpi['year'] = hpi['quarter'].str[:4].astype(int)
hpi['q_num'] = hpi['quarter'].str[-1].astype(int)
hpi

,quarter,index,year,q_num
0,1990-Q1,24.3,1990,1
1,1990-Q2,24.4,1990,2
2,1990-Q3,25.0,1990,3
3,1990-Q4,24.7,1990,4
4,1991-Q1,24.9,1991,1
...,...,...,...,...
122,2020-Q3,133.9,2020,3
123,2020-Q4,138.1,2020,4
124,2021-Q1,142.2,2021,1
125,2021-Q2,146.4,2021,2


# Merge Price Index on Resale Prices

In [9]:
# rsp['q_num'] = rsp['month'].dt.quarter
# rsp['year'] = rsp['month'].dt.year
# rsp

,id,month,block,town,street_name,address,storey_range,storey_range_min,storey_range_max,flat_model,...,remaining_lease,remaining_lease_years,remaining_lease_months,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr,q_num,year
0,1990_01_309_ANG_MO_KIO_AVE_1_10_TO_12,1990-01-01,309,ANG MO KIO,ANG MO KIO AVE 1,309 ANG MO KIO AVENUE 1,10 TO 12,10,12,IMPROVED,...,NaN,86,<NA>,9000.0,88.490320,290.322581,1.028957,3.375844,1,1990
2004,1990_01_307_CLEMENTI_AVE_4_01_TO_03,1990-01-01,307,CLEMENTI,CLEMENTI AVE 4,307 CLEMENTI AVENUE 4,01 TO 03,1,3,NEW GENERATION,...,NaN,88,<NA>,47000.0,213.814919,701.492537,2.429715,7.971506,1,1990
2005,1990_01_309_CLEMENTI_AVE_4_04_TO_06,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,309 CLEMENTI AVENUE 4,04 TO 06,4,6,NEW GENERATION,...,NaN,89,<NA>,57000.0,211.873164,695.121951,2.380597,7.810359,1,1990
2006,1990_01_309_CLEMENTI_AVE_4_10_TO_12,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,309 CLEMENTI AVENUE 4,10 TO 12,10,12,NEW GENERATION,...,NaN,89,<NA>,34000.0,154.674622,507.462687,1.737917,5.701828,1,1990
2007,1990_01_310_CLEMENTI_AVE_4_04_TO_06,1990-01-01,310,CLEMENTI,CLEMENTI AVE 4,310 CLEMENTI AVENUE 4,04 TO 06,4,6,NEW GENERATION,...,NaN,89,<NA>,47000.0,213.814919,701.492537,2.402415,7.881939,1,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210529,2022_03_501B_YISHUN_ST_51_07_TO_09,2022-03-01,501B,YISHUN,YISHUN ST 51,501B YISHUN STREET 51,07 TO 09,7,9,Improved,...,94 years 11 months,95,11,652000.0,1774.371372,5821.428571,18.677593,61.278195,1,2022
210528,2022_03_820_YISHUN_ST_81_07_TO_09,2022-03-01,820,YISHUN,YISHUN ST 81,820 YISHUN STREET 81,07 TO 09,7,9,Improved,...,65 years 06 months,65,6,585000.0,1461.540937,4795.081967,22.485245,73.770492,1,2022
210527,2022_03_359_YISHUN_RING_RD_01_TO_03,2022-03-01,359,YISHUN,YISHUN RING RD,359 YISHUN RING ROAD,01 TO 03,1,3,Apartment,...,65 years 05 months,65,5,682000.0,1433.610989,4703.448276,22.055554,72.360743,1,2022
210538,2022_03_751_YISHUN_ST_72_04_TO_06,2022-03-01,751,YISHUN,YISHUN ST 72,751 YISHUN STREET 72,04 TO 06,4,6,New Generation,...,61 years 02 months,61,2,458000.0,1534.048303,5032.967033,25.148333,82.507656,1,2022


In [10]:
# rsp = rsp.merge(hpi[['q_num', 'year', 'index']], on=['year', 'q_num'])
# rsp

,id,month,block,town,street_name,address,storey_range,storey_range_min,storey_range_max,flat_model,...,remaining_lease_years,remaining_lease_months,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr,q_num,year,index
0,1990_01_309_ANG_MO_KIO_AVE_1_10_TO_12,1990-01-01,309,ANG MO KIO,ANG MO KIO AVE 1,309 ANG MO KIO AVENUE 1,10 TO 12,10,12,IMPROVED,...,86,<NA>,9000.0,88.490320,290.322581,1.028957,3.375844,1,1990,24.3
1,1990_01_307_CLEMENTI_AVE_4_01_TO_03,1990-01-01,307,CLEMENTI,CLEMENTI AVE 4,307 CLEMENTI AVENUE 4,01 TO 03,1,3,NEW GENERATION,...,88,<NA>,47000.0,213.814919,701.492537,2.429715,7.971506,1,1990,24.3
2,1990_01_309_CLEMENTI_AVE_4_04_TO_06,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,309 CLEMENTI AVENUE 4,04 TO 06,4,6,NEW GENERATION,...,89,<NA>,57000.0,211.873164,695.121951,2.380597,7.810359,1,1990,24.3
3,1990_01_309_CLEMENTI_AVE_4_10_TO_12,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,309 CLEMENTI AVENUE 4,10 TO 12,10,12,NEW GENERATION,...,89,<NA>,34000.0,154.674622,507.462687,1.737917,5.701828,1,1990,24.3
4,1990_01_310_CLEMENTI_AVE_4_04_TO_06,1990-01-01,310,CLEMENTI,CLEMENTI AVE 4,310 CLEMENTI AVENUE 4,04 TO 06,4,6,NEW GENERATION,...,89,<NA>,47000.0,213.814919,701.492537,2.402415,7.881939,1,1990,24.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1512236,2021_09_723_YISHUN_ST_71_10_TO_12,2021-09-01,723,YISHUN,YISHUN ST 71,723 YISHUN STREET 71,10 TO 12,10,12,Maisonette,...,63,9,740000.0,1544.876663,5068.493151,24.521852,80.452272,3,2021,150.6
1512237,2021_09_828_YISHUN_ST_81_07_TO_09,2021-09-01,828,YISHUN,YISHUN ST 81,828 YISHUN STREET 81,07 TO 09,7,9,Apartment,...,65,5,732000.0,1571.222485,5154.929577,24.172654,79.306609,3,2021,150.6
1512238,2021_09_826_YISHUN_ST_81_07_TO_09,2021-09-01,826,YISHUN,YISHUN ST 81,826 YISHUN STREET 81,07 TO 09,7,9,Maisonette,...,65,4,760000.0,1586.630086,5205.479452,24.409694,80.084299,3,2021,150.6
1512239,2021_09_315B_YISHUN_AVE_9_04_TO_06,2021-09-01,315B,YISHUN,YISHUN AVE 9,315B YISHUN AVENUE 9,04 TO 06,4,6,Improved,...,92,1,558888.0,1520.973723,4990.071429,16.532323,54.239907,3,2021,150.6


In [11]:
# rsp['resale_price_norm'] = rsp['resale_price'] / rsp['index'] * 100
# rsp

,id,month,block,town,street_name,address,storey_range,storey_range_min,storey_range_max,flat_model,...,remaining_lease_months,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr,q_num,year,index,resale_price_norm
0,1990_01_309_ANG_MO_KIO_AVE_1_10_TO_12,1990-01-01,309,ANG MO KIO,ANG MO KIO AVE 1,309 ANG MO KIO AVENUE 1,10 TO 12,10,12,IMPROVED,...,<NA>,9000.0,88.490320,290.322581,1.028957,3.375844,1,1990,24.3,37037.037037
1,1990_01_307_CLEMENTI_AVE_4_01_TO_03,1990-01-01,307,CLEMENTI,CLEMENTI AVE 4,307 CLEMENTI AVENUE 4,01 TO 03,1,3,NEW GENERATION,...,<NA>,47000.0,213.814919,701.492537,2.429715,7.971506,1,1990,24.3,193415.637860
2,1990_01_309_CLEMENTI_AVE_4_04_TO_06,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,309 CLEMENTI AVENUE 4,04 TO 06,4,6,NEW GENERATION,...,<NA>,57000.0,211.873164,695.121951,2.380597,7.810359,1,1990,24.3,234567.901235
3,1990_01_309_CLEMENTI_AVE_4_10_TO_12,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,309 CLEMENTI AVENUE 4,10 TO 12,10,12,NEW GENERATION,...,<NA>,34000.0,154.674622,507.462687,1.737917,5.701828,1,1990,24.3,139917.695473
4,1990_01_310_CLEMENTI_AVE_4_04_TO_06,1990-01-01,310,CLEMENTI,CLEMENTI AVE 4,310 CLEMENTI AVENUE 4,04 TO 06,4,6,NEW GENERATION,...,<NA>,47000.0,213.814919,701.492537,2.402415,7.881939,1,1990,24.3,193415.637860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1512236,2021_09_723_YISHUN_ST_71_10_TO_12,2021-09-01,723,YISHUN,YISHUN ST 71,723 YISHUN STREET 71,10 TO 12,10,12,Maisonette,...,9,740000.0,1544.876663,5068.493151,24.521852,80.452272,3,2021,150.6,491367.861886
1512237,2021_09_828_YISHUN_ST_81_07_TO_09,2021-09-01,828,YISHUN,YISHUN ST 81,828 YISHUN STREET 81,07 TO 09,7,9,Apartment,...,5,732000.0,1571.222485,5154.929577,24.172654,79.306609,3,2021,150.6,486055.776892
1512238,2021_09_826_YISHUN_ST_81_07_TO_09,2021-09-01,826,YISHUN,YISHUN ST 81,826 YISHUN STREET 81,07 TO 09,7,9,Maisonette,...,4,760000.0,1586.630086,5205.479452,24.409694,80.084299,3,2021,150.6,504648.074369
1512239,2021_09_315B_YISHUN_AVE_9_04_TO_06,2021-09-01,315B,YISHUN,YISHUN AVE 9,315B YISHUN AVENUE 9,04 TO 06,4,6,Improved,...,1,558888.0,1520.973723,4990.071429,16.532323,54.239907,3,2021,150.6,371107.569721


In [14]:
# price_columns = ['resale_price', 'price_per_sq_ft', 'price_per_sq_m',
#                  'price_per_sq_ft_per_lease_yr', 'price_per_sq_m_per_lease_yr']
#
# for col in price_columns:
#     rsp[f"{col}_norm"] = rsp[col] / rsp['index'] * 100


In [4]:
rsp = rsp[['id', 'month', 'block', 'town', 'street_name',
     'storey_range', 'storey_range_min', 'storey_range_max', 'flat_model',
     'flat_type', 'n_rooms', 'floor_area_sqm',
     'lease_commence_date', 'remaining_lease', 'remaining_lease_years', 'remaining_lease_months',
     'resale_price',
     'price_per_sq_ft',
     'price_per_sq_m',
     'price_per_sq_ft_per_lease_yr',
     'price_per_sq_m_per_lease_yr',
     ]]

In [5]:
rsp.dtypes

id                                      object
month                           datetime64[ns]
block                                   object
town                                    object
street_name                             object
storey_range                            object
storey_range_min                         int64
storey_range_max                         int64
flat_model                              object
flat_type                               object
n_rooms                                  Int64
floor_area_sqm                         float64
lease_commence_date                      int64
remaining_lease                         object
remaining_lease_years                    int64
remaining_lease_months                   Int64
resale_price                           float64
price_per_sq_ft                        float64
price_per_sq_m                         float64
price_per_sq_ft_per_lease_yr           float64
price_per_sq_m_per_lease_yr            float64
dtype: object

In [6]:
rsp

,id,month,block,town,street_name,storey_range,storey_range_min,storey_range_max,flat_model,flat_type,...,floor_area_sqm,lease_commence_date,remaining_lease,remaining_lease_years,remaining_lease_months,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr
0,1990_01_309_ANG_MO_KIO_AVE_1_10_TO_12,1990-01-01,309,ANG MO KIO,ANG MO KIO AVE 1,10 TO 12,10,12,IMPROVED,1 ROOM,...,31.0,1977,NaN,86,<NA>,9000.0,88.490320,290.322581,1.028957,3.375844
2004,1990_01_307_CLEMENTI_AVE_4_01_TO_03,1990-01-01,307,CLEMENTI,CLEMENTI AVE 4,01 TO 03,1,3,NEW GENERATION,3 ROOM,...,67.0,1979,NaN,88,<NA>,47000.0,213.814919,701.492537,2.429715,7.971506
2005,1990_01_309_CLEMENTI_AVE_4_04_TO_06,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,04 TO 06,4,6,NEW GENERATION,3 ROOM,...,82.0,1980,NaN,89,<NA>,57000.0,211.873164,695.121951,2.380597,7.810359
2006,1990_01_309_CLEMENTI_AVE_4_10_TO_12,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,10 TO 12,10,12,NEW GENERATION,3 ROOM,...,67.0,1980,NaN,89,<NA>,34000.0,154.674622,507.462687,1.737917,5.701828
2007,1990_01_310_CLEMENTI_AVE_4_04_TO_06,1990-01-01,310,CLEMENTI,CLEMENTI AVE 4,04 TO 06,4,6,NEW GENERATION,3 ROOM,...,67.0,1980,NaN,89,<NA>,47000.0,213.814919,701.492537,2.402415,7.881939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210529,2022_03_501B_YISHUN_ST_51_07_TO_09,2022-03-01,501B,YISHUN,YISHUN ST 51,07 TO 09,7,9,Improved,5 ROOM,...,112.0,2018,94 years 11 months,95,11,652000.0,1774.371372,5821.428571,18.677593,61.278195
210528,2022_03_820_YISHUN_ST_81_07_TO_09,2022-03-01,820,YISHUN,YISHUN ST 81,07 TO 09,7,9,Improved,5 ROOM,...,122.0,1988,65 years 06 months,65,6,585000.0,1461.540937,4795.081967,22.485245,73.770492
210527,2022_03_359_YISHUN_RING_RD_01_TO_03,2022-03-01,359,YISHUN,YISHUN RING RD,01 TO 03,1,3,Apartment,EXECUTIVE,...,145.0,1988,65 years 05 months,65,5,682000.0,1433.610989,4703.448276,22.055554,72.360743
210538,2022_03_751_YISHUN_ST_72_04_TO_06,2022-03-01,751,YISHUN,YISHUN ST 72,04 TO 06,4,6,New Generation,4 ROOM,...,91.0,1984,61 years 02 months,61,2,458000.0,1534.048303,5032.967033,25.148333,82.507656


In [7]:
engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

sql_alc_cnxn = engine.connect()

rsp.to_sql('resale_prices_full', sql_alc_cnxn, if_exists='replace', index=False)

sql_alc_cnxn.close()

# Exploring Duplicate Issue

## resale_prices_full

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from MyCreds.mycreds import Capstone_AWS_PG
from sqlalchemy import create_engine

In [2]:
sql = """select * from resale_prices_full"""

engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

with engine.connect() as cnxn:
    df = pd.read_sql(sql, cnxn)

In [3]:
df.shape

(1524524, 21)

In [4]:
df.columns

Index(['id', 'month', 'block', 'town', 'street_name', 'storey_range',
       'storey_range_min', 'storey_range_max', 'flat_model', 'flat_type',
       'n_rooms', 'floor_area_sqm', 'lease_commence_date', 'remaining_lease',
       'remaining_lease_years', 'remaining_lease_months', 'resale_price',
       'price_per_sq_ft', 'price_per_sq_m', 'price_per_sq_ft_per_lease_yr',
       'price_per_sq_m_per_lease_yr'],
      dtype='object')

In [14]:
id_counts = df.value_counts('id').reset_index().rename({0: 'n_appearnaces'}, axis=1)
id_counts[id_counts['n_appearnaces'] > 1]

,id,n_appearnaces
0,1997_08_10_JLN_BATU_01_TO_03,32
1,1997_08_11_JLN_BATU_01_TO_03,30
2,2009_02_5_CHANGI_VILLAGE_RD_01_TO_03,24
3,2009_01_5_CHANGI_VILLAGE_RD_01_TO_03,24
4,2009_02_5_CHANGI_VILLAGE_RD_04_TO_06,18
...,...,...
613003,1999_01_547_BEDOK_NTH_ST_3_13_TO_15,2
613004,1999_01_562_CHOA_CHU_KANG_ST_52_04_TO_06,2
613005,1998_12_820_YISHUN_ST_81_01_TO_03,2
613006,1999_01_562_ANG_MO_KIO_AVE_3_13_TO_15,2


In [8]:
dups = df.groupby(df.columns.tolist(), as_index=False).size()
dups[dups['size'] > 1].sort_values('id')

,id,month,block,town,street_name,storey_range,storey_range_min,storey_range_max,flat_model,flat_type,...,lease_commence_date,remaining_lease,remaining_lease_years,remaining_lease_months,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr,size
840,2015_01_57_GEYLANG_BAHRU_16_TO_18,2015-01-01,57,KALLANG/WHAMPOA,GEYLANG BAHRU,16 TO 18,16,18,Improved,3 ROOM,...,1974,58,58,0.0,315000.0,1477.107645,4846.153846,25.467373,83.554377,2
1715,2015_02_403_TAMPINES_ST_41_07_TO_09,2015-02-01,403,TAMPINES,TAMPINES ST 41,07 TO 09,7,9,Improved,3 ROOM,...,1985,69,69,0.0,350000.0,1546.086907,5072.463768,22.407057,73.513968,2
4768,2015_04_701_BEDOK_RESERVOIR_RD_10_TO_12,2015-04-01,701,BEDOK,BEDOK RESERVOIR RD,10 TO 12,10,12,New Generation,4 ROOM,...,1980,64,64,0.0,400000.0,1310.967700,4301.075269,20.483870,67.204301,2
4792,2015_04_714_CLEMENTI_WEST_ST_2_10_TO_12,2015-04-01,714,CLEMENTI,CLEMENTI WEST ST 2,10 TO 12,10,12,New Generation,3 ROOM,...,1980,64,64,0.0,300000.0,1364.776076,4477.611940,21.324626,69.962687,2
5776,2015_05_409_BEDOK_NTH_AVE_2_07_TO_09,2015-05-01,409,BEDOK,BEDOK NTH AVE 2,07 TO 09,7,9,New Generation,3 ROOM,...,1980,63,64,0.0,340000.0,1523.999951,5000.000000,23.812499,78.125000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141173,2021_12_817C_KEAT_HONG_LINK_10_TO_12,2021-12-01,817C,CHOA CHU KANG,KEAT HONG LINK,10 TO 12,10,12,Improved,5 ROOM,...,2017,94 years 10 months,94,10.0,655000.0,1782.535657,5848.214286,18.963145,62.215046,2
141230,2021_12_879B_TAMPINES_AVE_8_10_TO_12,2021-12-01,879B,TAMPINES,TAMPINES AVE 8,10 TO 12,10,12,Model A,4 ROOM,...,2017,94 years 07 months,94,7.0,610000.0,1999.225742,6559.139785,21.268359,69.778083,2
143224,2022_01_676A_JURONG_WEST_ST_64_07_TO_09,2022-01-01,676A,JURONG WEST,JURONG WEST ST 64,07 TO 09,7,9,Premium Apartment,5 ROOM,...,2008,85 years 06 months,85,6.0,508000.0,1394.940496,4576.576577,16.411065,53.842077,2
143412,2022_01_802C_KEAT_HONG_CL_04_TO_06,2022-01-01,802C,CHOA CHU KANG,KEAT HONG CL,04 TO 06,4,6,Model A,3 ROOM,...,2018,95 years 02 months,95,2.0,388000.0,1765.110391,5791.044776,18.580109,60.958366,2


In [9]:
df[df['id'].isin(dups[dups['size'] > 1]['id'])]

,id,month,block,town,street_name,storey_range,storey_range_min,storey_range_max,flat_model,flat_type,...,floor_area_sqm,lease_commence_date,remaining_lease,remaining_lease_years,remaining_lease_months,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr
1366542,2015_01_57_GEYLANG_BAHRU_16_TO_18,2015-01-01,57,KALLANG/WHAMPOA,GEYLANG BAHRU,16 TO 18,16,18,Improved,3 ROOM,...,65.0,1974,58,58,0.0,315000.0,1477.107645,4846.153846,25.467373,83.554377
1366558,2015_01_57_GEYLANG_BAHRU_16_TO_18,2015-01-01,57,KALLANG/WHAMPOA,GEYLANG BAHRU,16 TO 18,16,18,Improved,3 ROOM,...,65.0,1974,58,58,0.0,315000.0,1477.107645,4846.153846,25.467373,83.554377
1368061,2015_02_403_TAMPINES_ST_41_07_TO_09,2015-02-01,403,TAMPINES,TAMPINES ST 41,07 TO 09,7,9,Improved,3 ROOM,...,69.0,1985,69,69,0.0,350000.0,1546.086907,5072.463768,22.407057,73.513968
1368081,2015_02_403_TAMPINES_ST_41_07_TO_09,2015-02-01,403,TAMPINES,TAMPINES ST 41,07 TO 09,7,9,Improved,3 ROOM,...,69.0,1985,69,69,0.0,350000.0,1546.086907,5072.463768,22.407057,73.513968
1369772,2015_04_701_BEDOK_RESERVOIR_RD_10_TO_12,2015-04-01,701,BEDOK,BEDOK RESERVOIR RD,10 TO 12,10,12,New Generation,4 ROOM,...,93.0,1980,64,64,0.0,400000.0,1310.967700,4301.075269,20.483870,67.204301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520821,2022_01_676A_JURONG_WEST_ST_64_07_TO_09,2022-01-01,676A,JURONG WEST,JURONG WEST ST 64,07 TO 09,7,9,Premium Apartment,5 ROOM,...,111.0,2008,85 years 06 months,85,6.0,508000.0,1394.940496,4576.576577,16.411065,53.842077
1520860,2022_01_676A_JURONG_WEST_ST_64_07_TO_09,2022-01-01,676A,JURONG WEST,JURONG WEST ST 64,07 TO 09,7,9,Premium Apartment,5 ROOM,...,111.0,2008,85 years 06 months,85,6.0,508000.0,1394.940496,4576.576577,16.411065,53.842077
1523242,2022_02_664B_PUNGGOL_DR_13_TO_15,2022-02-01,664B,PUNGGOL,PUNGGOL DR,13 TO 15,13,15,Model A,4 ROOM,...,90.0,2016,93 years 05 months,93,5.0,540000.0,1828.799941,6000.000000,19.664515,64.516129
1523253,2022_02_664B_PUNGGOL_DR_13_TO_15,2022-02-01,664B,PUNGGOL,PUNGGOL DR,13 TO 15,13,15,Model A,4 ROOM,...,90.0,2016,93 years 04 months,93,4.0,520000.0,1761.066610,5777.777778,18.936200,62.126643


In [15]:
## resale_prices_norm

In [16]:
sql = """select * from resale_prices_norm"""


engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

with engine.connect() as cnxn:
    norm = pd.read_sql(sql, cnxn)

In [17]:
norm.shape

(1524524, 26)

In [52]:
len(norm['resale_prices_norm_id'].unique())

808654

In [55]:
norm.iloc[:, 1:].drop_duplicates()

,month,block,town,street_name,storey_range,storey_range_min,storey_range_max,flat_model,flat_type,n_rooms,...,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr,resale_price_norm,price_per_sq_ft_norm,price_per_sq_ft_per_lease_yr_norm,price_per_sq_m_norm,price_per_sq_m_per_lease_yr_norm,join_address
0,1990-01-01,309,ANG MO KIO,ANG MO KIO AVE 1,10 TO 12,10,12,IMPROVED,1 ROOM,1.0,...,88.490320,290.322581,1.028957,3.375844,55777.777778,548.421488,6.376994,1799.283154,20.921897,309 ANG MO KIO AVE 1
1,1990-01-01,307,CLEMENTI,CLEMENTI AVE 4,01 TO 03,1,3,NEW GENERATION,3 ROOM,3.0,...,213.814919,701.492537,2.429715,7.971506,291283.950617,1325.124557,15.058234,4347.521651,49.403655,307 CLEMENTI AVE 4
2,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,04 TO 06,4,6,NEW GENERATION,3 ROOM,3.0,...,211.873164,695.121951,2.380597,7.810359,353259.259259,1313.090473,14.753826,4308.039747,48.404941,309 CLEMENTI AVE 4
3,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,10 TO 12,10,12,NEW GENERATION,3 ROOM,3.0,...,154.674622,507.462687,1.737917,5.701828,210716.049383,958.600743,10.770795,3145.015662,35.337255,309 CLEMENTI AVE 4
4,1990-01-01,310,CLEMENTI,CLEMENTI AVE 4,04 TO 06,4,6,NEW GENERATION,3 ROOM,3.0,...,213.814919,701.492537,2.402415,7.881939,291283.950617,1325.124557,14.889040,4347.521651,48.848558,310 CLEMENTI AVE 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1524519,2022-03-01,501B,YISHUN,YISHUN ST 51,07 TO 09,7,9,Improved,5 ROOM,5.0,...,1774.371372,5821.428571,18.677593,61.278195,NaN,NaN,NaN,NaN,NaN,501B YISHUN ST 51
1524520,2022-03-01,820,YISHUN,YISHUN ST 81,07 TO 09,7,9,Improved,5 ROOM,5.0,...,1461.540937,4795.081967,22.485245,73.770492,NaN,NaN,NaN,NaN,NaN,820 YISHUN ST 81
1524521,2022-03-01,359,YISHUN,YISHUN RING RD,01 TO 03,1,3,Apartment,EXECUTIVE,NaN,...,1433.610989,4703.448276,22.055554,72.360743,NaN,NaN,NaN,NaN,NaN,359 YISHUN RING RD
1524522,2022-03-01,751,YISHUN,YISHUN ST 72,04 TO 06,4,6,New Generation,4 ROOM,4.0,...,1534.048303,5032.967033,25.148333,82.507656,NaN,NaN,NaN,NaN,NaN,751 YISHUN ST 72


In [19]:
norm.columns

Index(['resale_prices_norm_id', 'month', 'block', 'town', 'street_name',
       'storey_range', 'storey_range_min', 'storey_range_max', 'flat_model',
       'flat_type', 'n_rooms', 'floor_area_sqm', 'lease_commence_date',
       'remaining_lease', 'remaining_lease_years', 'remaining_lease_months',
       'resale_price', 'price_per_sq_ft', 'price_per_sq_m',
       'price_per_sq_ft_per_lease_yr', 'price_per_sq_m_per_lease_yr',
       'resale_price_norm', 'price_per_sq_ft_norm',
       'price_per_sq_ft_per_lease_yr_norm', 'price_per_sq_m_norm',
       'price_per_sq_m_per_lease_yr_norm'],
      dtype='object')

In [20]:
id_counts_norm = norm.value_counts('resale_prices_norm_id').reset_index().rename({0: 'n_appearnaces'}, axis=1)
id_counts_norm[id_counts_norm['n_appearnaces'] > 1]

,resale_prices_norm_id,n_appearnaces
0,1997_08_10_JLN_BATU_01_TO_03,32
1,1997_08_11_JLN_BATU_01_TO_03,30
2,2009_02_5_CHANGI_VILLAGE_RD_01_TO_03,24
3,2009_01_5_CHANGI_VILLAGE_RD_01_TO_03,24
4,2009_02_5_CHANGI_VILLAGE_RD_04_TO_06,18
...,...,...
613003,1999_01_547_BEDOK_NTH_ST_3_13_TO_15,2
613004,1999_01_562_CHOA_CHU_KANG_ST_52_04_TO_06,2
613005,1998_12_820_YISHUN_ST_81_01_TO_03,2
613006,1999_01_562_ANG_MO_KIO_AVE_3_13_TO_15,2


In [21]:
dups_norm = norm.groupby(norm.columns.tolist(), as_index=False).size()
dups_norm[dups_norm['size'] > 1].sort_values('resale_prices_norm_id')

,resale_prices_norm_id,month,block,town,street_name,storey_range,storey_range_min,storey_range_max,flat_model,flat_type,...,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr,resale_price_norm,price_per_sq_ft_norm,price_per_sq_ft_per_lease_yr_norm,price_per_sq_m_norm,price_per_sq_m_per_lease_yr_norm,size
840,2015_01_57_GEYLANG_BAHRU_16_TO_18,2015-01-01,57,KALLANG/WHAMPOA,GEYLANG BAHRU,16 TO 18,16,18,Improved,3 ROOM,...,1477.107645,4846.153846,25.467373,83.554377,349845.132743,1640.504508,28.284560,5382.232811,92.797117,2
1715,2015_02_403_TAMPINES_ST_41_07_TO_09,2015-02-01,403,TAMPINES,TAMPINES ST 41,07 TO 09,7,9,Improved,3 ROOM,...,1546.086907,5072.463768,22.407057,73.513968,388716.814159,1717.114220,24.885713,5633.577017,81.646044,2
4768,2015_04_701_BEDOK_RESERVOIR_RD_10_TO_12,2015-04-01,701,BEDOK,BEDOK RESERVOIR RD,10 TO 12,10,12,New Generation,4 ROOM,...,1310.967700,4301.075269,20.483870,67.204301,446222.222222,1462.457301,22.850895,4798.088411,74.970131,2
4792,2015_04_714_CLEMENTI_WEST_ST_2_10_TO_12,2015-04-01,714,CLEMENTI,CLEMENTI WEST ST 2,10 TO 12,10,12,New Generation,3 ROOM,...,1364.776076,4477.611940,21.324626,69.962687,334666.666667,1522.483533,23.788805,4995.024876,78.047264,2
5776,2015_05_409_BEDOK_NTH_AVE_2_07_TO_09,2015-05-01,409,BEDOK,BEDOK NTH AVE 2,07 TO 09,7,9,New Generation,3 ROOM,...,1523.999951,5000.000000,23.812499,78.125000,379288.888889,1700.106612,26.564166,5577.777778,87.152778,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133011,2021_09_310C_PUNGGOL_WALK_01_TO_03,2021-09-01,310C,PUNGGOL,PUNGGOL WALK,01 TO 03,1,3,Premium Apartment,4 ROOM,...,1673.086903,5489.130435,17.990182,59.022908,505000.000000,1673.086903,17.990182,5489.130435,59.022908,2
133039,2021_09_312_ANG_MO_KIO_AVE_3_01_TO_03,2021-09-01,312,ANG MO KIO,ANG MO KIO AVE 3,01 TO 03,1,3,Improved,5 ROOM,...,1432.312149,4699.186992,26.042039,85.439763,578000.000000,1432.312149,26.042039,4699.186992,85.439763,2
133049,2021_09_314B_PUNGGOL_WAY_04_TO_06,2021-09-01,314B,PUNGGOL,PUNGGOL WAY,04 TO 06,4,6,Premium Apartment,4 ROOM,...,1704.258010,5591.397849,18.130404,59.482956,520000.000000,1704.258010,18.130404,5591.397849,59.482956,2
133651,2021_09_524C_PASIR_RIS_ST_51_10_TO_12,2021-09-01,524C,PASIR RIS,PASIR RIS ST 51,10 TO 12,10,12,Model A,4 ROOM,...,1900.903165,6236.559140,20.661991,67.788686,580000.000000,1900.903165,20.661991,6236.559140,67.788686,2


In [22]:
norm[norm['resale_prices_norm_id'].isin(dups_norm[dups_norm['size'] > 1].sort_values('resale_prices_norm_id')['resale_prices_norm_id'])]

,resale_prices_norm_id,month,block,town,street_name,storey_range,storey_range_min,storey_range_max,flat_model,flat_type,...,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr,resale_price_norm,price_per_sq_ft_norm,price_per_sq_ft_per_lease_yr_norm,price_per_sq_m_norm,price_per_sq_m_per_lease_yr_norm
1366542,2015_01_57_GEYLANG_BAHRU_16_TO_18,2015-01-01,57,KALLANG/WHAMPOA,GEYLANG BAHRU,16 TO 18,16,18,Improved,3 ROOM,...,315000.0,1477.107645,4846.153846,25.467373,83.554377,349845.132743,1640.504508,28.284560,5382.232811,92.797117
1366558,2015_01_57_GEYLANG_BAHRU_16_TO_18,2015-01-01,57,KALLANG/WHAMPOA,GEYLANG BAHRU,16 TO 18,16,18,Improved,3 ROOM,...,315000.0,1477.107645,4846.153846,25.467373,83.554377,349845.132743,1640.504508,28.284560,5382.232811,92.797117
1368061,2015_02_403_TAMPINES_ST_41_07_TO_09,2015-02-01,403,TAMPINES,TAMPINES ST 41,07 TO 09,7,9,Improved,3 ROOM,...,350000.0,1546.086907,5072.463768,22.407057,73.513968,388716.814159,1717.114220,24.885713,5633.577017,81.646044
1368081,2015_02_403_TAMPINES_ST_41_07_TO_09,2015-02-01,403,TAMPINES,TAMPINES ST 41,07 TO 09,7,9,Improved,3 ROOM,...,350000.0,1546.086907,5072.463768,22.407057,73.513968,388716.814159,1717.114220,24.885713,5633.577017,81.646044
1369772,2015_04_701_BEDOK_RESERVOIR_RD_10_TO_12,2015-04-01,701,BEDOK,BEDOK RESERVOIR RD,10 TO 12,10,12,New Generation,4 ROOM,...,400000.0,1310.967700,4301.075269,20.483870,67.204301,446222.222222,1462.457301,22.850895,4798.088411,74.970131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511146,2021_09_314B_PUNGGOL_WAY_04_TO_06,2021-09-01,314B,PUNGGOL,PUNGGOL WAY,04 TO 06,4,6,Premium Apartment,4 ROOM,...,520000.0,1704.258010,5591.397849,18.130404,59.482956,520000.000000,1704.258010,18.130404,5591.397849,59.482956
1511149,2021_09_217D_SUMANG_WALK_13_TO_15,2021-09-01,217D,PUNGGOL,SUMANG WALK,13 TO 15,13,15,Model A,4 ROOM,...,518000.0,1716.156467,5630.434783,18.256984,59.898242,518000.000000,1716.156467,18.256984,5630.434783,59.898242
1511152,2021_09_217D_SUMANG_WALK_13_TO_15,2021-09-01,217D,PUNGGOL,SUMANG WALK,13 TO 15,13,15,Model A,4 ROOM,...,518000.0,1716.156467,5630.434783,18.256984,59.898242,518000.000000,1716.156467,18.256984,5630.434783,59.898242
1511220,2021_09_310C_PUNGGOL_WALK_01_TO_03,2021-09-01,310C,PUNGGOL,PUNGGOL WALK,01 TO 03,1,3,Premium Apartment,5 ROOM,...,600000.0,1632.857091,5357.142857,17.557603,57.603687,600000.000000,1632.857091,17.557603,5357.142857,57.603687


## resale_location_features

In [25]:
sql = """select * from sg_buildings_location_features"""


engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

with engine.connect() as cnxn:
    sg = pd.read_sql(sql, cnxn)

In [26]:
sg.shape

(142886, 347)

In [33]:
norm['join_address'] = norm['block'].astype(str) + ' ' + norm['street_name'].astype(str)
sg['join_address'] = sg['blk_no'].astype(str) + ' ' + sg['short_r_name'].astype(str)

norm.merge(sg, on='join_address', how='left')

,resale_prices_norm_id,month,block,town,street_name,storey_range,storey_range_min,storey_range_max,flat_model,flat_type,...,dist_to_con_area_id_239,dist_to_con_area_id_240,dist_to_con_area_id_241,dist_to_con_area_id_242,dist_to_con_area_id_243,dist_to_con_area_id_244,dist_to_con_area_id_245,dist_to_con_area_id_246,dist_to_con_area_id_247,dist_to_con_area_id_248
0,1990_01_309_ANG_MO_KIO_AVE_1_10_TO_12,1990-01-01,309,ANG MO KIO,ANG MO KIO AVE 1,10 TO 12,10,12,IMPROVED,1 ROOM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990_01_307_CLEMENTI_AVE_4_01_TO_03,1990-01-01,307,CLEMENTI,CLEMENTI AVE 4,01 TO 03,1,3,NEW GENERATION,3 ROOM,...,5055.625835,6855.178054,7580.918891,10244.425068,5941.870234,6311.130467,5962.357214,10295.245331,9040.381671,9335.190456
2,1990_01_309_CLEMENTI_AVE_4_04_TO_06,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,04 TO 06,4,6,NEW GENERATION,3 ROOM,...,5039.415827,6877.134084,7634.803032,10299.894667,6012.653144,6379.967374,6037.579608,10358.135878,9101.052647,9376.821441
3,1990_01_309_CLEMENTI_AVE_4_10_TO_12,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,10 TO 12,10,12,NEW GENERATION,3 ROOM,...,5039.415827,6877.134084,7634.803032,10299.894667,6012.653144,6379.967374,6037.579608,10358.135878,9101.052647,9376.821441
4,1990_01_310_CLEMENTI_AVE_4_04_TO_06,1990-01-01,310,CLEMENTI,CLEMENTI AVE 4,04 TO 06,4,6,NEW GENERATION,3 ROOM,...,4995.399700,6864.595002,7652.761602,10319.259788,6049.176524,6414.213486,6079.310657,10385.403357,9126.005477,9382.340748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1926478,2022_03_820_YISHUN_ST_81_07_TO_09,2022-03-01,820,YISHUN,YISHUN ST 81,07 TO 09,7,9,Improved,5 ROOM,...,15683.802906,14994.140923,13057.173667,14043.042450,11211.598049,11412.258469,10804.743868,13180.667720,12877.996597,14951.835718
1926479,2022_03_359_YISHUN_RING_RD_01_TO_03,2022-03-01,359,YISHUN,YISHUN RING RD,01 TO 03,1,3,Apartment,EXECUTIVE,...,17635.213651,16791.687127,14733.321002,15486.861690,13002.962729,13176.727971,12598.594016,14574.006142,14386.362877,16538.350935
1926480,2022_03_751_YISHUN_ST_72_04_TO_06,2022-03-01,751,YISHUN,YISHUN ST 72,04 TO 06,4,6,New Generation,4 ROOM,...,17215.047008,16588.501609,14662.552267,15625.391112,12809.048556,13013.226089,12402.072729,14752.175445,14470.198662,16553.837691
1926481,2022_03_751_YISHUN_ST_72_04_TO_06,2022-03-01,751,YISHUN,YISHUN ST 72,04 TO 06,4,6,New Generation,4 ROOM,...,17215.990979,16588.689451,14662.269619,15624.425527,12809.156960,13013.233960,12402.183251,14751.070576,14469.399118,16553.287653


In [34]:
len(sg['join_address'].unique())

133290

In [35]:
len(norm['join_address'].unique())

9518

In [36]:
len(norm['join_address'].isin(sg['join_address']))

1524524

In [37]:
# sg['join_address'].value_counts()

address_counts_sg= sg.value_counts('join_address').reset_index().rename({0: 'n_appearnaces'}, axis=1)
address_counts_sg[address_counts_sg['n_appearnaces'] > 1]

,join_address,n_appearnaces
0,500 DOVER RD,132
1,21 TAMPINES AVE 1,82
2,535 CLEMENTI RD,80
3,50 NANYANG AVE,68
4,1 MOUNT PLEASANT RD,58
...,...,...
6200,1000 UPP CHANGI RD NTH,2
6201,33A SWISS CLUB RD,2
6202,201A COMPASSVALE DR,2
6203,55 FLORISSA PK,2


In [38]:
sg

,sg_buildings_location_features_id,address,blk_no,building,latitude,longitude,longtitude,postal,road_name,short_r_name,...,dist_to_con_area_id_240,dist_to_con_area_id_241,dist_to_con_area_id_242,dist_to_con_area_id_243,dist_to_con_area_id_244,dist_to_con_area_id_245,dist_to_con_area_id_246,dist_to_con_area_id_247,dist_to_con_area_id_248,join_address
0,118,20 REPUBLIC AVENUE OCBC NICOLL HIGHWAY MRT STA...,20,OCBC NICOLL HIGHWAY MRT STATION,1.299587,103.864040,103.864040,38970,REPUBLIC AVENUE,REPUBLIC AVE,...,6059.347311,3756.446604,1816.802323,5022.351892,4683.348179,5075.305839,942.400600,1807.237816,3644.091416,20 REPUBLIC AVE
1,0,101A BAYFRONT AVENUE TEMPORARY SITE OFFICE SIN...,101A,TEMPORARY SITE OFFICE,1.275697,103.855652,103.855652,18895,BAYFRONT AVENUE,BAYFRONT AVE,...,4698.611071,3472.336797,1280.648451,5607.264711,5243.410236,5839.706885,2191.996829,1809.059589,1832.377169,101A BAYFRONT AVE
2,1,1 STRAITS BOULEVARD SINGAPORE CHINESE CULTURAL...,1,SINGAPORE CHINESE CULTURAL CENTRE,1.275829,103.849576,103.849576,18906,STRAITS BOULEVARD,STRAITS BOULEVARD,...,4023.602031,2948.064284,1296.158736,5144.809340,4787.128765,5407.289369,2294.937659,1601.665849,1162.333975,1 STRAITS BOULEVARD
3,2,11A STRAITS BOULEVARD TEMPORARY SITE OFFICE SI...,11A,TEMPORARY SITE OFFICE,1.274950,103.851665,103.851665,18907,STRAITS BOULEVARD,STRAITS BOULEVARD,...,4264.445108,3183.564726,1341.734859,5369.673931,5008.956154,5623.701677,2327.157978,1735.354855,1410.053853,11A STRAITS BOULEVARD
4,3,2 CENTRAL BOULEVARD IOI CENTRAL BOULEVARD TOWE...,2,IOI CENTRAL BOULEVARD TOWERS,1.279777,103.851513,103.851513,18916,CENTRAL BOULEVARD,CENTRAL BOULEVARD,...,4224.178971,2834.639505,817.910211,4962.733697,4598.936423,5197.159347,1813.778805,1215.677136,1372.737276,2 CENTRAL BOULEVARD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142881,142721,23 PONGGOL TWENTY-FOURTH AVENUE PONGGOL PARK S...,23,PONGGOL PARK,1.418939,103.908908,103.908908,829493,PONGGOL TWENTY-FOURTH AVENUE,PONGGOL TWENTY-FOURTH AVE,...,18758.073747,16244.432449,15810.303374,15337.504057,15338.490052,14979.161428,14798.871568,15137.537830,17404.006863,23 PONGGOL TWENTY-FOURTH AVE
142882,142722,25 PONGGOL TWENTY-FOURTH AVENUE SINGAPORE 829494,25,NIL,1.418914,103.909043,103.909043,829494,PONGGOL TWENTY-FOURTH AVENUE,PONGGOL TWENTY-FOURTH AVE,...,18764.308089,16250.193761,15813.699759,15345.052577,15345.672999,14986.842778,14802.294340,15141.980410,17408.229852,25 PONGGOL TWENTY-FOURTH AVE
142883,142723,27 PONGGOL TWENTY-FOURTH AVENUE PONGGOL PARK S...,27,PONGGOL PARK,1.418928,103.909291,103.909291,829495,PONGGOL TWENTY-FOURTH AVENUE,PONGGOL TWENTY-FOURTH AVE,...,18781.252196,16266.396697,15826.068797,15364.024780,15364.089234,15006.016112,14814.709260,15156.069355,17421.956923,27 PONGGOL TWENTY-FOURTH AVE
142884,142724,29 PONGGOL TWENTY-FOURTH AVENUE SINGAPORE 829496,29,NIL,1.419180,103.909259,103.909259,829496,PONGGOL TWENTY-FOURTH AVENUE,PONGGOL TWENTY-FOURTH AVE,...,18802.221259,16287.979222,15850.274095,15383.096089,15383.715302,15024.879661,14838.887695,15179.203388,17445.330265,29 PONGGOL TWENTY-FOURTH AVE


In [45]:
match_cols = sg[sg['join_address'].isin(norm['join_address'].values)][['blk_no', 'short_r_name']]
match_cols

,blk_no,short_r_name
262,34,UPP CROSS ST
878,4,SAGO LANE
880,32,NEW MKT RD
1686,1,TG PAGAR PLAZA
1687,2,TG PAGAR PLAZA
...,...,...
142756,617D,PUNGGOL DR
142757,618D,PUNGGOL DR
142758,619D,PUNGGOL DR
142760,637D,PUNGGOL DR


In [46]:
match_cols['full_address'] = match_cols['blk_no'].astype(str) + ' ' + match_cols['short_r_name']
match_cols.value_counts('full_address')

full_address
1 TG PAGAR PLAZA         10
849 YISHUN RING RD        6
221 BOON LAY PL           6
210 NEW UPP CHANGI RD     5
413 YISHUN RING RD        5
                         ..
337 WOODLANDS AVE 1       1
337 UBI AVE 1             1
337 TAMPINES ST 32        1
135 TECK WHYE LANE        1
403 FAJAR RD              1
Length: 9390, dtype: int64

In [49]:
sg[sg['join_address'] == '138 TAMPINES ST 11']

,sg_buildings_location_features_id,address,blk_no,building,latitude,longitude,longtitude,postal,road_name,short_r_name,...,dist_to_con_area_id_240,dist_to_con_area_id_241,dist_to_con_area_id_242,dist_to_con_area_id_243,dist_to_con_area_id_244,dist_to_con_area_id_245,dist_to_con_area_id_246,dist_to_con_area_id_247,dist_to_con_area_id_248,join_address
63568,79417,138 TAMPINES STREET 11 SINGAPORE 521138,138,NIL,1.345505,103.945588,103.945588,521138,TAMPINES STREET 11,TAMPINES ST 11,...,16428.249952,13976.339889,12125.902158,14374.867708,14156.998474,14196.887676,11339.371070,12150.262257,13986.953357,138 TAMPINES ST 11
63569,79418,138 TAMPINES STREET 11 DBS NTUC TAMPINES STREE...,138,DBS NTUC TAMPINES STREET 11,1.345281,103.945344,103.945344,521138,TAMPINES STREET 11,TAMPINES ST 11,...,16392.927442,13941.597207,12089.786948,14342.315268,14124.100982,14164.792741,11303.677868,12114.898716,13950.821144,138 TAMPINES ST 11
63570,79419,138 TAMPINES STREET 11 DBS TAMPINES STREET 11 ...,138,DBS TAMPINES STREET 11,1.345281,103.945344,103.945344,521138,TAMPINES STREET 11,TAMPINES ST 11,...,16392.927442,13941.597207,12089.786948,14342.315268,14124.100982,14164.792741,11303.677868,12114.898716,13950.821144,138 TAMPINES ST 11
63571,79420,138 TAMPINES STREET 11 OCBC TAMPINES STREET 11...,138,OCBC TAMPINES STREET 11 - 7 ELEVEN,1.345320,103.945385,103.945385,521138,TAMPINES STREET 11,TAMPINES ST 11,...,16398.829387,13947.396514,12095.831417,14347.732435,14129.577988,14170.130659,11309.645847,12120.807807,13956.868766,138 TAMPINES ST 11


In [51]:
sg['building'].value_counts()

NIL                                  47885
SERANGOON GARDEN ESTATE               2648
SELETAR HILLS ESTATE                  1265
FRANKEL ESTATE                        1261
SENNETT ESTATE                        1181
                                     ...  
OCBC ALJUNIED MRT                        1
UOB ALJUNIED MRT STATION                 1
GREEN AVENUE HOME FOR THE ELDERLY        1
HIGHPOINT SOCIAL ENTERPRISE ARK          1
RIVER OF LIFE COMMUNITY CHURCH           1
Name: building, Length: 16378, dtype: int64